In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## Client ကို ဖန်တီးခြင်း

ဤနမူနာတွင် [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) ကို အသုံးပြု၍ LLM ကို အသုံးပြုမည်ဖြစ်သည်။

`model` ကို `gpt-4o-mini` ဟု သတ်မှတ်ထားသည်။ GitHub Models စျေးကွက်တွင် ရရှိနိုင်သော အခြားမော်ဒယ်တစ်ခုကို ပြောင်းလဲကြည့်ပြီး မတူညီသောရလဒ်များကို ကြည့်ရှုနိုင်ပါသည်။

အမြန်စမ်းသပ်မှုအနေနှင့် `What is the capital of France` ဟု ရိုးရိုးရှင်းရှင်း prompt တစ်ခုကို လုပ်ဆောင်ကြည့်မည်။


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## အလုပ်ဆောင်ပုံများကို သတ်မှတ်ခြင်း

ဤဥပမာတွင်၊ အေးဂျင့်တစ်ဦးအား ရရှိနိုင်သော အပန်းဖြေခရီးစဉ်နေရာများနှင့် ၎င်းတို့ရရှိနိုင်မှုစာရင်းပါဝင်သည့် အလုပ်ဆောင်ပုံတစ်ခုကို အသုံးပြုနိုင်ရန် ခွင့်ပြုမည်ဖြစ်သည်။

ဤအခြေအနေကို ခရီးသွားအေးဂျင့်တစ်ဦးက ခရီးသွားဒေတာဘေ့စ်တစ်ခုကို အသုံးပြုခွင့်ရရှိထားသော အခြေအနေတစ်ခုအဖြစ် စဉ်းစားနိုင်သည်။

ဤနမူနာကို လေ့လာသည့်အခါ၊ အေးဂျင့်က ခေါ်ယူနိုင်မည့် အလုပ်ဆောင်ပုံအသစ်များနှင့် ကိရိယာအသစ်များကို သတ်မှတ်ကြည့်ရန် လွတ်လပ်စွာ ကြိုးစားနိုင်ပါသည်။


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## Function Tool ကိုသတ်မှတ်ခြင်း  
`vacation_destinations` ကို `FunctionTool` အဖြစ် အသုံးပြုစေလိုပါက၊ ၎င်းကို Function Tool အဖြစ် သတ်မှတ်ရန် လိုအပ်ပါသည်။

ထို့အပြင်၊ အသုံးပြုသူ၏ တောင်းဆိုမှုနှင့် ပတ်သက်ပြီး အဆိုပါ tool ကို ဘာအတွက် အသုံးပြုရမည်ကို အေးဂျင့်အနေနှင့် သဘောပေါက်စေရန် အထောက်အကူဖြစ်စေမည့် ဖော်ပြချက်တစ်ရပ်ကိုလည်း ပံ့ပိုးပေးပါမည်။


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## အေးဂျင့်ကို သတ်မှတ်ခြင်း

အောက်ပါကုဒ်တွင် အေးဂျင့်ကို ဖန်တီးနိုင်ပါပြီ။ `system_message` ကို သတ်မှတ်ပြီး အသုံးပြုသူများအား အပန်းဖြေခရီးစဉ်နေရာများ ရှာဖွေရာတွင် အကူအညီပေးရန် ညွှန်ကြားချက်များ ပေးလိုက်ပါသည်။

ထို့အပြင် `reflect_on_tool_use` parameter ကို true အဖြစ် သတ်မှတ်ထားပါသည်။ ဒါကတော့ Tool ကို ခေါ်ယူပြီးရရှိသော အဖြေကို သဘာဝဘာသာစကားဖြင့် ပြန်လည်ပေးပို့ရန် LLM ကို အသုံးပြုခွင့်ပေးသည်။

အဆိုပါ parameter ကို false အဖြစ် သတ်မှတ်ပြီး ကွာခြားချက်ကို ကြည့်ရှုနိုင်ပါသည်။


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## အေးဂျင့်ကို အလုပ်လုပ်စေခြင်း

အခုတော့ တိုကျိုကို ခရီးသွားဖို့ မေးမြန်းတဲ့ မူလအသုံးပြုသူ၏ မက်ဆေ့ချ်နှင့်အတူ အေးဂျင့်ကို အလုပ်လုပ်စေနိုင်ပါပြီ။

ဒီမြို့ကို ခရီးသွားရန်နေရာကို ပြောင်းလဲပြီး အေးဂျင့်က မြို့ရရှိနိုင်မှုကို ဘယ်လိုတုံ့ပြန်မယ်ဆိုတာ ကြည့်ရှုနိုင်ပါတယ်။


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**အကြောင်းကြားချက်**:  
ဤစာရွက်စာတမ်းကို AI ဘာသာပြန်ဝန်ဆောင်မှု [Co-op Translator](https://github.com/Azure/co-op-translator) ကို အသုံးပြု၍ ဘာသာပြန်ထားပါသည်။ ကျွန်ုပ်တို့သည် တိကျမှုအတွက် ကြိုးစားနေသော်လည်း၊ အလိုအလျောက် ဘာသာပြန်မှုများတွင် အမှားများ သို့မဟုတ် မတိကျမှုများ ပါဝင်နိုင်သည်ကို သတိပြုပါ။ မူရင်းစာရွက်စာတမ်းကို ၎င်း၏ မူလဘာသာစကားဖြင့် အာဏာတရားရှိသော အရင်းအမြစ်အဖြစ် ရှုလေ့လာသင့်ပါသည်။ အရေးကြီးသော အချက်အလက်များအတွက် လူက ဘာသာပြန်မှု ဝန်ဆောင်မှုကို အသုံးပြုရန် အကြံပြုပါသည်။ ဤဘာသာပြန်မှုကို အသုံးပြုခြင်းမှ ဖြစ်ပေါ်လာသော အလွဲအလွတ်များ သို့မဟုတ် အနားလွဲမှုများအတွက် ကျွန်ုပ်တို့သည် တာဝန်မယူပါ။
